In [1]:
!pip uninstall spacy


Uninstalling spacy-2.2.4:
  Would remove:
    /usr/local/bin/spacy
    /usr/local/lib/python3.6/dist-packages/bin/*
    /usr/local/lib/python3.6/dist-packages/spacy-2.2.4.dist-info/*
    /usr/local/lib/python3.6/dist-packages/spacy/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/bin/theano_cache.py
    /usr/local/lib/python3.6/dist-packages/bin/theano_nose.py
Proceed (y/n)? y
  Successfully uninstalled spacy-2.2.4


In [2]:
!pip install spacy


     |████████████████████████████████| 10.4MB 7.6MB/s 
     |████████████████████████████████| 1.1MB 51.2MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0


In [3]:
from __future__ import unicode_literals, print_function, division
from keras.preprocessing.text import text_to_word_sequence
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from google.colab import drive
import pandas as pd
drive.mount('/content/drive', force_remount=True)
from collections import defaultdict
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np
import torchtext
from torchtext.data import Field, Dataset, Example
from nltk.translate.bleu_score import sentence_bleu,SmoothingFunction
import spacy
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

Mounted at /content/drive


In [4]:
!python -m spacy download xx_ent_wiki_sm
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('xx_ent_wiki_sm')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [5]:
en = spacy.load('en_core_web_sm')


In [149]:
contractions = { 
"i ain't": "i am not",
"ain't that": "is not that",
"ain't she": "is not she",
"ain't it": "is not it",
"it ain't": "it is not",
"that ain't": "that is not",
"this ain't": "this is not",
"ain't you": "are not you",
"aren't": "you are",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'll": "I will",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"o'clock": "of the clock",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there's": "there is",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who's": "who is",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"would've": "would have",
"wouldn't": "would not",
"y'all": "you all",
"you'd": "you would",
"you'll": "you will",
"you're": "you are",
"you've": "you have"

}

contractions2 = {
    "'s": " is",
    "ain't" : " is not",
    "'ve": "have",
    "'d": "would",
    "'ll": "will"
}



In [150]:
learning_rate= 0.0004
hidden_size = 256
emb_size = 300

In [151]:
def preprocess_en (X) :
    for k,v in contractions.items():
        X = X.lower().replace(k,v)
    for k,v in contractions2.items():
        X = X.lower().replace(k,v)
    X = X.strip().split('\n')
    #X = [re.sub(r'[^\w\s]', '', xx) for xx in X]
    return X 
    

def preprocess_hi (X) :
    for k,v in contractions_hin.items():
        X = X.replace(k,v)
    X = X.strip().split('\n')
    return X

def preprocess2(X):
    cont={
    "<eos>":"",
    "<s>":" "
    }
    for k,v in cont.items():
      X=X.replace(k,v)
    return X

In [152]:
def create_dictionary(X) :
    word_freq = defaultdict(int)

    for i in range(len(X)):
        X[i] = [w for w in X[i] if w!=''] 
        for word in X[i] :
            word_freq[word] += 1

    word_i = {"<sos>": 0, "<eos>": 1}
    i_word = {0: "<sos>", 1: "<eos>"}
    wid = 2
    for w,c in word_freq.items():
        word_i[w] = wid 
        i_word[wid] = w
        wid += 1
    return wid,word_i,i_word

In [153]:
def reverse_en(x) :
    words = x.split()
    words = list(reversed(words))
    words = (" ".join(words))
    return words

In [154]:
def delete_min(query,document,summary):
  query = [xx.split(" ") for xx in query]
  summary=[xx.split(" ") for xx in summary]
  document=[xx.split(" ") for xx in document]
  for i in range(len(query)) :
    if (len(query[i]) and len(summary[i])) :
        continue
    else :
        del query[i]
        del summary[i]
        del document[i]

  return query,document,summary

In [155]:
def summary_creation(reference_list,query,document,summary):
  for i in  range(len(query)):
    query[i]=query[i].strip()
    if query[i] in reference_lists.keys():
      reference_lists[query[i]].append(summary[i])
    else:
      reference_lists[query[i]]=[]
      reference_lists[query[i]].append(summary[i])
    
  return reference_lists

In [156]:


train_query=open('/content/drive/MyDrive/data/train_query').read()
test_query=open('/content/drive/MyDrive/data/test_query').read()
val_query=open('/content/drive/MyDrive/data/valid_query').read()



train_content=open('/content/drive/MyDrive/data/train_content').read()
test_content=open('/content/drive/MyDrive/data/test_content').read()
val_content=open('/content/drive/MyDrive/data/valid_content').read()


train_summary=open('/content/drive/MyDrive/data/train_summary').read()
test_summary=open('/content/drive/MyDrive/data/test_summary').read()
val_summary=open('/content/drive/MyDrive/data/valid_summary').read()



train_query=preprocess_en(train_query)
train_content=preprocess_en(train_content)
train_summary=preprocess_en(train_summary)


test_query=preprocess_en(test_query)
test_content=preprocess_en(test_content)
test_summary=preprocess_en(test_summary)

val_query=preprocess_en(val_query)
val_content=preprocess_en(val_content)
val_summary=preprocess_en(val_summary)
#y_raw = preprocess_en(y_raw)

reference_lists={}
# reference_lists[query[0]]=summary[0]
sum=0
total_queries=[]


refermcer_lists=summary_creation(reference_lists,train_query,train_content,train_summary)
reference_lists=summary_creation(reference_lists,test_query,test_content,test_summary)
reference_lists=summary_creation(reference_lists,val_query,val_content,val_summary)
print(test_query[220])
train_query,train_content,train_summary=delete_min(train_query,train_content,train_summary)
test_query,test_content,test_summary=delete_min(test_query,test_content,test_summary)
val_query,val_content,val_summary=delete_min(val_query,val_content,val_summary)


<s> abstinence message : is the abstinence-only message appropriate ? <eos>


In [157]:
def tokenize_en(text):
  ret = [tok.text for tok in en.tokenizer(str(text))]
  return ret
def tokenize_hin(text):
  ret = [tok.text for tok in hin.tokenizer(str(text))]
  return ret

In [158]:
TEXT_en = torchtext.data.Field(
  tokenize    = tokenize_en,
  lower       = True,
  init_token  = '<sos>',
  eos_token   = '<eos>'
)
TEXT_hin = torchtext.data.Field(
  tokenize    = tokenize_hin,
  lower       = True,
  init_token  = '<sos>',
  eos_token   = '<eos>'
)

In [159]:
fields = [('query',TEXT_en),('document',TEXT_en),('summary',TEXT_en)]
def fun_bucket(query,content,summary): 
  examples = []
  for i in range(len(query)):
    temp = Example.fromlist([query[i],content[i],summary[i]],fields=fields)
    examples.append(temp)
  return examples

In [160]:
train_check = torchtext.data.Dataset(fun_bucket(train_query,train_content,train_summary),fields)
test_check=torchtext.data.Dataset(fun_bucket(test_query,test_content,test_summary),fields)
val_check=torchtext.data.Dataset(fun_bucket(val_query,val_content,val_summary),fields)
print(train_check[0])

In [161]:
batch_size = 8

train_iterator = torchtext.data.BucketIterator(
    train_check,batch_size,sort_within_batch=False,device=device,repeat=False,sort_key=False
)
test_iterator = torchtext.data.BucketIterator(
    test_check,batch_size,sort_within_batch=False,device=device,repeat=False,sort_key=False
)
val_iterator = torchtext.data.BucketIterator(
    val_check,batch_size,sort_within_batch=False,device=device,repeat=False,sort_key=False
)

In [162]:
TEXT_en.build_vocab(train_check,test_check,val_check)
print(TEXT_en.vocab)
#TEXT_hin.build_vocab(train_check,test_check)


In [163]:
vocab_en = TEXT_en.vocab 
print(vocab_en)
#vocab_hin = TEXT_hin.vocab
print(len(vocab_en.stoi))
src_vocab_size = len(vocab_en.stoi)
trg_vocab_size=len(vocab_en.stoi)

29253


# Encoder RNN

In [164]:
class query_encoderRNN(nn.Module):

  def __init__(self,input_size,emb_size,hidden_size,layers=1):
    super(query_encoderRNN,self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.n_layers = layers
    self.embedding = nn.Embedding(input_size,emb_size)
    self.rnn = nn.GRU(emb_size,hidden_size,num_layers=4,bidirectional=True)
    self.dropout = nn.Dropout(0.5)
  
  def forward(self,input):
    embedding = self.dropout(self.embedding(input))
    outputs,hidden = self.rnn(embedding)
    hidden=hidden[3]
    hidden=hidden.unsqueeze(1)
    
    hidden=hidden.permute(1,0,2)
    #print(hidden.shape)
    return outputs,hidden


In [238]:
class document_encoderRNN(nn.Module):

  def __init__(self,input_size,emb_size,hidden_size,layers=1):
    super(document_encoderRNN,self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.n_layers = layers
    self.embedding = nn.Embedding(input_size,emb_size)
    self.rnn = nn.GRU(emb_size,hidden_size,num_layers=4,bidirectional=True)
    self.dropout = nn.Dropout(0.5)
  
  
  def forward(self,input,query_encoder):
    embedding = self.dropout(query_encoder.embedding(input))
    outputs,hidden = self.rnn(embedding)
    
    hidden=torch.cat((hidden[3],hidden[2]),1)
    print(hidden.shape,"docu")
    hidden=hidden.unsqueeze(1)
    hidden=hidden.permute(1,0,2)
    print("encoder",hidden.shape,outputs.shape)
    return outputs,hidden


In [247]:
class decoderRNN(nn.Module):
    def __init__(self, output_size, embedding_size, hidden_size):
        super(decoderRNN, self).__init__()
        
        self.batch_size = batch_size
        self.hidden_size =hidden_size
        self.output_size = output_size
        self.embedding_size = embedding_size
        
        self.gru = nn.GRU(self.embedding_size + self.hidden_size*2, self.hidden_size,batch_first=True,num_layers=4,bidirectional=True)
        
        self.dropout = nn.Dropout(0.5)
        self.hidden_size =2* hidden_size
        # used for attention
        self.fc = nn.Linear(self.hidden_size, self.output_size) 
        self.Wq = nn.Linear(self.hidden_size, self.hidden_size)
        self.Wd = nn.Linear(self.hidden_size, self.hidden_size)
        self.Uq = nn.Linear(self.hidden_size*2,self.hidden_size)
        self.Ud = nn.Linear(self.hidden_size*2,self.hidden_size)
        self.Vq = nn.Linear(self.hidden_size, 1)
        self.Vd = nn.Linear(self.hidden_size,1)
        self.Z  = nn.Linear(self.hidden_size*2,self.hidden_size)
        self.Wdec=nn.Linear(self.hidden_size,self.hidden_size)
        self.Vdec=nn.Linear(self.hidden_size,self.hidden_size)
        self.Wo=nn.Linear(self.hidden_size,trg_vocab_size)
    def forward(self,yt_,dt_,hq,hd,query_encoder):
        #print(dt_.shape)
        dt_=dt_.permute(1,0,2)
        hq = hq.permute(1,0,2)
        hd = hd.permute(1,0,2)
        print("gh")
        yt_=query_encoder.embedding(yt_).unsqueeze(1)
        #print(yt_.shape,dt_.shape)
        x = torch.cat((yt_,dt_), -1)
       
        output, st = self.gru(x)
        output =  output.view(-1, output.size(2))
        yt = self.fc(output)
        st=st[3]
        st=st.unsqueeze(1)
        score = torch.tanh(self.Wq(st) + self.Uq(hq))
        attention_weights = torch.softmax(self.Vq(score), dim=1)
        context_vector = attention_weights * hq
        qt = torch.sum(context_vector, dim=1)
        qt=qt.unsqueeze(1)
       
        score = torch.tanh(self.Wd(st) + self.Ud(hd)+self.Z(qt))
        attention_weights = torch.softmax(self.Vd(score), dim=1)
        context_vector = attention_weights * hd
        dt = torch.sum(context_vector, dim=1)
        #yt=torch.softmax(self.Wo(self.Wdec(st).squeeze() + self.Vdec(dt)),dim=1)
        dt=dt.unsqueeze(1)
        dt=dt.permute(1,0,2)
        #yt=yt.argmax(1)
        #print("hello,",yt.shape)
        #zt=torch.sum(yt,dim=1)
        print(dt.shape)
        #return yt, dt, attention_weights

In [248]:
t_f_c = 0.5

class Seq2Seq(nn.Module):
  def __init__(self,query_encoder,document_encoder,decoder,device):
    super(Seq2Seq,self).__init__()
    self.query_encoder = query_encoder
    self.document_encoder = document_encoder
    self.decoder=decoder
    self.device = device
  
  def forward(self,query_source,document_source,target,teaching = 0.5):
    
    target_length = target.shape[0]
    batch_size = target.shape[1]
    outputs = torch.zeros(target_length,batch_size,trg_vocab_size).to(self.device)
    query_encoder_outputs,query_hidden = self.query_encoder(query_source)
    document_encoder_outputs,document_hidden=self.document_encoder(document_source,self.query_encoder)
    input = target[0,:]
    wts = []
    
    hidden=document_hidden
    print(hidden.shape)
    for i in range(1,target_length):
      output , hidden,_ = self.decoder(input,hidden,query_encoder_outputs,document_encoder_outputs,self.query_encoder)
      #print(output.shape)
      outputs[i]=output
      top1 = output.argmax(1)
      input = top1 if random.random()<t_f_c else target[i]
      wts.append(_)
    #print(outputs.shape)
    return outputs, wts


In [249]:
dropout = 0.5
query_encoder = query_encoderRNN(len(vocab_en),emb_size, hidden_size).to(device)
document_encoder=document_encoderRNN(len(vocab_en),emb_size,hidden_size).to(device)
decoder = decoderRNN(len(vocab_en),emb_size,hidden_size).to(device)

model = Seq2Seq(query_encoder,document_encoder,decoder,device).to(device)

In [250]:
def train(model,iterator,query_encoder_optimizer,document_encoder_optimizer,decoder_optimizer,criterion,clip=1):
    model.train()
    total_loss = 0
    for i,batch in enumerate(iterator):
        query_source = batch.query
        document_source=batch.document
        target = batch.summary
        query_encoder_optimizer.zero_grad()
        document_encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        output,_ = model(query_source ,document_source,target)
        
        output_dimension = output.shape[-1]
      
        output = output[1:].view(-1 ,output_dimension).to(device)
        target = target[1:].view(-1).to(device)
        #print("hi ",output.shape,target.shape)
        #print("namasete")
        loss = criterion(output,target)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        query_encoder_optimizer.step()
        decoder_optimizer.step()
        document_encoder_optimizer.step()
        total_loss +=loss.item()
    return total_loss/len(iterator)


In [251]:

def evaluate(model,iterator,criterion):
  model.eval()
  epoch_loss = 0
  with torch.no_grad():
    for i,batch in enumerate(iterator):
      query_source = batch.query
      document_source=batch.document
      target = batch.summary
      #print(query_source,document_source,target)
      output,_ = model(query_source,document_source,target,0)
      output_dim = output.shape[-1]
      output = output[1:].view(-1, output_dim).to(device)

      target = target[1:].view(-1).to(device)
      
      loss = criterion(output,target)
      epoch_loss+=loss.item()
  return epoch_loss/len(iterator)


In [252]:
import time
EPOCH = 1
timr = []
train_loss_list = [] 
test_loss_list = []
learning_rate =  0.0004

In [253]:
query_encoder_optimizer = optim.SGD(query_encoder.parameters(), lr=learning_rate)
document_encoder_optimizer=optim.SGD(document_encoder.parameters(),lr=learning_rate)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss(ignore_index = 0)

In [254]:

EPOCH = 50
for epoch in range(EPOCH):
    start_time = time.time()
    train_loss = train(model,train_iterator,query_encoder_optimizer,document_encoder_optimizer,decoder_optimizer,criterion)
    test_loss = evaluate(model,val_iterator,criterion)
    test_rouge()
    PATH = '/content/drive/My Drive/baseline'
    torch.save(model.state_dict(), PATH)
    model = Seq2Seq(query_encoder,document_encoder,decoder,device).to(device)
    model.load_state_dict(torch.load('/content/drive/My Drive/baseline'))
    end_time = time.time()
    train_loss_list.append(train_loss)
    test_loss_list.append(test_loss)
    timr.append(end_time-start_time)
    print(epoch,train_loss,test_loss)



torch.Size([8, 512]) docu
encoder torch.Size([1, 8, 512]) torch.Size([140, 8, 512])
torch.Size([1, 8, 512])
gh


RuntimeError: ignored

In [ ]:
'''
EPOCH = 200

for epoch in range(EPOCH):
    start_time = time.time()
    train_loss = train(model,train_iterator,query_encoder_optimizer,document_encoder_optimizer,decoder_optimizer,criterion)
    test_loss = evaluate(model,test_iterator,criterion)
    end_time = time.time()
    train_loss_list.append(train_loss)
    test_loss_list.append(test_loss)
    timr.append(end_time-start_time)
    print(epoch,train_loss,test_loss)
  '''

In [ ]:
'''
EPOCH = 300

for epoch in range(EPOCH):
    start_time = time.time()
    train_loss = train(model,train_iterator,query_encoder_optimizer,document_encoder_optimizer,decoder_optimizer,criterion)
    test_loss = evaluate(model,test_iterator,criterion)
    end_time = time.time()
    train_loss_list.append(train_loss)
    test_loss_list.append(test_loss)
    timr.append(end_time-start_time)
    print(epoch,train_loss,test_loss)
'''

In [ ]:
print(summary[0])

In [ ]:
  # TOTAL TESTING  CODE
def test_rouge():
  model = Seq2Seq(query_encoder,document_encoder,decoder,device).to(device)
  model.load_state_dict(torch.load('/content/drive/My Drive/baseline'))
  model.eval()
  !pip install rouge/requirements.txt
  !pip install rouge-score
  from rouge_score import rouge_scorer
  results = []
  rouge1=0
  rouge2=0
  rougel=0
  c=0
  results_query = []
  results_pred = []
  results_actual_summary = []
  results_document = []
  with torch.no_grad():
    for j,batch in enumerate(test_iterator):
      query_source = batch.query
      document_source=batch.document
      target = batch.summary
      output,bat_wts = model(query_source,document_source,target,1)
      output_dim = output.shape[0]
      output = output.permute(1,0,2)
      wts = []
      for i in range(1,target.shape[0]):
          l1 = [[p.item() for p in t] for t in bat_wts[i-1]]
          l2 = [[row[i] for row in l1] for i in range(len(l1[0]))]
          #print(l2)
          wts.append(l2)
      attn_wts = np.array(wts).T

      for t in range(query_source.shape[1]):

        target_pred = ""
        target_actual = ""
        query_actual = ""
        document_actual=""
        actual = []
        pred = []
        for i in range(query_source.shape[0]):
          query_actual += vocab_en.itos[query_source[i,t]]+" "
        query_actual = query_actual.replace('<pad>','').replace('<sos>','').replace('<eos>','').strip()
        temp = query_actual.split()
        query_actual = ' '.join([str(elem) for elem in temp])
        for i in range(document_source.shape[0]):
          document_actual += vocab_en.itos[document_source[i,t]]+" "
        document_actual = document_actual.replace('<pad>','').replace('<sos>','<s>').strip()
        temp = document_actual.split()
        document_actual = ' '.join([str(elem) for elem in temp])
        
        for i in range(1,target.shape[0]):
          target_actual += vocab_en.itos[target[i,t]]+" "

        target_actual = target_actual.replace('<pad>','').replace('<sos>','').replace('<eos>','').strip()
        output_temp = output[t]
        for i in range(1,target.shape[0]):
          ind = output_temp.argmax(1)[i].item()
          target_pred += vocab_en.itos[ind]+" "
          
          if ind == 1 :
            break
        
        #results_attn_wts.append(attn_wts[t][:,0:i])
        target_pred = target_pred.replace('<pad>','').replace('<sos>','<s>').strip()
        query_actual=query_actual+' '+'<eos>'
        query_actual=query_actual.strip()
        
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL','rouge2'], use_stemmer=True)
        #print(query_actual)
        r1_score=0
        r2_score=0
        rl_score=0
        if query_actual  in reference_lists.keys():
          for i in reference_lists[query_actual]:
            scores=scorer.score(i,target_pred)
            r1_score=max(r1_score,scores['rouge1'].fmeasure)
            r2_score=max(r2_score,scores['rouge2'].fmeasure)
            rl_score=max(rl_score,scores['rougeL'].fmeasure)
            c=c+1
            #print(scores)
          rouge1=rouge1+r1_score
          rouge2=rouge2+r2_score
          rougel=rougel+rl_score

        results_query.append(query_actual)
        results_document.append(document_actual)
        results_actual_summary.append(target_actual)
        results_pred.append(target_pred)
        print("rouge1",rouge1/967)
        print("rouge2",rouge2/967)
        print("rougel",rougel/967)


In [ ]:
print(rouge1,rouge2,rougel,c)

In [ ]:
print(rouge1/(len(test_iterator)*batch_size))
print(rouge2/(len(test_iterator)*batch_size))
print(rougel/(len(test_iterator)*batch_size))

In [ ]:
PATH = '/content/drive/My Drive/baseline'
torch.save(model.state_dict(), PATH)
import pickle
 
favorite_color = { "lion": vocab_en} 
pickle.dump( favorite_color, open( "/content/drive/My Drive/baseline_dict", "wb" ) )

In [ ]:
print(test_loss_list)
print(train_loss_list)
print(len(train_loss_list))

In [ ]:
filename = '/content/drive/My Drive/results_baseline_update123.csv'

In [ ]:
import csv  
    
# field names  
names = ['Query', 'Actual_Summary','Predicted_Summary']  
    
# data rows of csv file  
rows = list(zip(results_query,  results_actual_summary,results_pred))
    
# writing to csv file  
with open(filename, 'w') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)  
        
    # writing the fields  
    csvwriter.writerow(names)
        
    # writing the data rows  
    csvwriter.writerows(rows) 
print("hi")